In [9]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

driver = webdriver.Chrome()

state_bus_urls = [
    "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/kaac-transport",
    "https://www.redbus.in/online-booking/jksrtc"
    
]
routedata = []

for state_bus_url in state_bus_urls:
    driver.get(state_bus_url)

    wait = WebDriverWait(driver, 10)
    time.sleep(5) # Let the user actually see something!   

    def route_page():
        routescontainer = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//a[@class="route"]')))
        
        for route in routescontainer:
            routename = route.get_attribute('title')
            routelink = route.get_attribute('href')
            routedata.append({'routename': routename, 'routelink': routelink})
        
            driver.implicitly_wait(3)

    for page_number in range(1, 6):
        
        print(f"Scraping page {page_number}")
        route_page()
        if page_number > 0:
            try:
                pagination_container = wait.until(EC.presence_of_element_located(
                    (By.XPATH, '//*[@class="DC_117_paginationTable"]')
                ))
                next_page_button = pagination_container.find_element(
                    By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
                )
                actions = ActionChains(driver)
                actions.move_to_element(next_page_button).perform()
                time.sleep(3) 
    
                print(f"Clicking on page {page_number + 1}")
                next_page_button.click()
                wait.until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
            except (NoSuchElementException):
                print("Successfully scrapped all route_links in all pages of this state respective to state_bus_url")
                break

driver.quit()

df = pd.DataFrame(routedata)
df

Scraping page 1
Clicking on page 2
Successfully navigated to page 2
Scraping page 2
Successfully scrapped all route_links in all pages of this state respective to state_bus_url
Scraping page 1
Clicking on page 2
Successfully navigated to page 2
Scraping page 2
Clicking on page 3
Successfully navigated to page 3
Scraping page 3
Successfully scrapped all route_links in all pages of this state respective to state_bus_url
Scraping page 1
Clicking on page 2
Successfully navigated to page 2
Scraping page 2
Clicking on page 3
Successfully navigated to page 3
Scraping page 3
Successfully scrapped all route_links in all pages of this state respective to state_bus_url
Scraping page 1
Clicking on page 2
Successfully navigated to page 2
Scraping page 2
Clicking on page 3
Successfully navigated to page 3
Scraping page 3
Clicking on page 4
Successfully navigated to page 4
Scraping page 4
Clicking on page 5
Successfully navigated to page 5
Scraping page 5
Successfully scrapped all route_links in all 

,routename,routelink
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
...,...,...
414,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...
415,Jammu (j and k) to Kishtwar,https://www.redbus.in/bus-tickets/jammu-to-kis...
416,Jammu (j and k) to Mendhar (J & K),https://www.redbus.in/bus-tickets/jammu-to-men...
417,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...


In [15]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains


driver = webdriver.Chrome()

bus_details = []

for item in routedata:
    route_link = item['routelink']
    route_name =item['routename']
    driver.get(route_link)

    time.sleep(3)

    try:
        govt_bus = driver.find_element(By.XPATH, "//div[@class='button']")
        govt_bus.click()
        time.sleep(2)
    except (NoSuchElementException):
        print(f"Govt bus not available for this route {route_name}")
    
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        time.sleep(1)       
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)
               
        new_page_source = driver.page_source
        time.sleep(2)    
        if new_page_source == old_page_source:
            scrolling = False
    
    def scrape_bus_details(driver):
        try:
            ul_element = driver.find_element(By.CSS_SELECTOR, "ul.bus-items")
            li_elements = ul_element.find_elements(By.CSS_SELECTOR, "li.row-sec")
              
            for route in li_elements:
                busname = route.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                bustype = route.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                departingtime = route.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                duration = route.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                reachingtime = route.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
                starrating = route.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                if starrating:
                    for rating in starrating:
                        starrating = rating.text
                else:
                    starrating = 0.0        
                price = route.find_element(By.XPATH, ".//span[@class='f-19 f-bold' or @class='f-bold f-19']").text
                seatavailability = route.find_element(By.XPATH, ".//div[@class='seat-left m-top-30' or @class='seat-left m-top-16']").text
                
                bus_details.append({
                    "Bus_route" : route_name,
                    "Bus_link" : route_link,
                    "Bus_name" : busname,
                    "Bus_type" : bustype,
                    "Departing_time" : departingtime,
                    "Duration" : duration,
                    "Reaching_time" : reachingtime,
                    "star_rating" : starrating,
                    "Price" : price,
                    "Seat_availability" : seatavailability
                })

        except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
            print(f"Bus Not Available in Route {route_name}")
    
    scrape_bus_details(driver)

driver.quit()

data = pd.DataFrame(bus_details)

data.to_csv('RB_bus_details.csv', index=False)

Govt bus not available for this route Coimbatore to Ooty
Bus Not Available in Route Coimbatore to Ooty
Govt bus not available for this route Hyderabad to Vijayawada
Govt bus not available for this route Hyderabad to Srisailam
Govt bus not available for this route Hyderabad to Godavarikhani
Govt bus not available for this route Hyderabad to Anantapur (andhra pradesh)
Govt bus not available for this route Hyderabad to Warangal
Govt bus not available for this route Hyderabad to Guntur (Andhra Pradesh)
Govt bus not available for this route Patiala to Delhi
Govt bus not available for this route Delhi to Patiala
Govt bus not available for this route Delhi to Amritsar
Govt bus not available for this route Amritsar to Delhi
Govt bus not available for this route Delhi Airport to Patiala
Govt bus not available for this route Amritsar to Delhi Airport
Govt bus not available for this route Kapurthala to Delhi
Govt bus not available for this route Chandigarh to Bathinda
Govt bus not available for t

In [17]:
df.to_csv('RB_route_links.csv', index=False)